In [34]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [42]:
train_data = pd.read_csv('data/tweets_lemm_train.csv')
train_data.head()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


In [43]:
valid_data = pd.read_csv('data/tweets_lemm_test.csv')
valid_data.head()

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [64]:
corpus = train_data['lemm_text']

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

count_tfidf = TfidfVectorizer(stop_words=stopwords)
features_tf_idf = count_tfidf.fit_transform(corpus)

print("Размер матрицы:", features_tf_idf.shape)

Размер матрицы: (5000, 9737)


[nltk_data] Downloading package stopwords to /Users/mike/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
target = train_data['positive']

train_features, test_features, train_target, test_target = train_test_split(features_tf_idf, target,
                                                                            test_size=0.2, random_state=42)

In [66]:
lg_model = LogisticRegression()
lg_model.fit(train_features, train_target)

train_predict = lg_model.predict(train_features)
print('train', accuracy_score(train_target, train_predict))

test_predict = lg_model.predict(test_features)
print('test', accuracy_score(test_target, test_predict))

train 0.90825
test 0.628


In [67]:
lg_CV_model = LogisticRegressionCV(cv=20, scoring='accuracy', max_iter=500)
lg_CV_model.fit(train_features, train_target)

train_predict = lg_CV_model.predict(train_features)
print('train', accuracy_score(train_target, train_predict))

test_predict = lg_CV_model.predict(test_features)
print('test', accuracy_score(test_target, test_predict))

train 0.9505
test 0.634


In [68]:
test_features

<1000x9737 sparse matrix of type '<class 'numpy.float64'>'
	with 6338 stored elements in Compressed Sparse Row format>

In [69]:
valid_corpus = valid_data['lemm_text']

# nltk.download('stopwords')
# stopwords = set(nltk_stopwords.words('russian'))

# count_tfidf = TfidfVectorizer(stop_words=stopwords)
valid_features_tf_idf = count_tfidf.transform(valid_corpus)
print("Размер матрицы:", valid_features_tf_idf.shape)

Размер матрицы: (3000, 9737)


In [70]:
lg_CV_model.predict(valid_features_tf_idf)

array([0, 1, 0, ..., 0, 0, 1])

In [72]:
valid_data['positive'] = lg_CV_model.predict(valid_features_tf_idf)

valid_data.to_csv('result')